### Week 6, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [22]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown
import os
from openai import AsyncOpenAI

load_dotenv(override=True)

gemini_client = AsyncOpenAI(api_key=os.getenv("GEMINI_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

gemini_model = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash", 
    openai_client=gemini_client
)

In [23]:
params = {"command": "uv", "args": ["run", "/home/vlad/projects/agents/6_mcp/mcp_weather/weather_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

In [24]:
mcp_tools

[Tool(name='get_temperature', title=None, description='Get the temperature for a given location.\n    \n    Args:\n        location: The location to get the temperature for\n    ', inputSchema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_temperatureArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'get_temperatureOutput', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=None),
 Tool(name='get_humidity', title=None, description='Get the humidity for a given location.\n    \n    Args:\n        location: The location to get the humidity for\n    ', inputSchema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_humidityArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'titl

In [25]:
instructions = "You are able to find information about the current weather based on the provided location."
request = "What is the current weather for Munich"
model = gemini_model

In [26]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="weather_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("weather_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current weather for Munich is:
Temperature: 3.5°C
Humidity: 89%
Pressure: 951.0 hPa
Wind speed: 6.5 km/h
Description: Moderate drizzle

In [2]:
from accounts import Account

In [3]:
account = Account.get("Vlad")
account

Account(name='vlad', balance=9585.172, strategy='', holdings={'AMZN': 9}, transactions=[3 shares of AMZN at 30.06 each., 3 shares of AMZN at 44.088 each., 3 shares of AMZN at 64.128 each.], portfolio_value_time_series=[('2025-12-05 10:32:51', 10074.82), ('2025-12-05 10:32:53', 10059.82), ('2025-12-05 10:37:41', 10377.556), ('2025-12-05 10:37:44', 10206.172), ('2025-12-05 10:37:45', 10206.172)])

In [4]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "vlad", "balance": 9540.082, "strategy": "", "holdings": {"AMZN": 12}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 30.06, "timestamp": "2025-12-05 10:32:51", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 44.088, "timestamp": "2025-12-05 10:37:41", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 64.128, "timestamp": "2025-12-05 10:37:44", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 15.03, "timestamp": "2025-12-05 11:54:47", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-05 10:32:51", 10074.82], ["2025-12-05 10:32:53", 10059.82], ["2025-12-05 10:37:41", 10377.556], ["2025-12-05 10:37:44", 10206.172], ["2025-12-05 10:37:45", 10206.172], ["2025-12-05 11:54:47", 9720.082]], "total_portfolio_value": 

In [5]:
account.report()

'{"name": "vlad", "balance": 9540.082, "strategy": "", "holdings": {"AMZN": 12}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 30.06, "timestamp": "2025-12-05 10:32:51", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 44.088, "timestamp": "2025-12-05 10:37:41", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 64.128, "timestamp": "2025-12-05 10:37:44", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 15.03, "timestamp": "2025-12-05 11:54:47", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-05 10:32:51", 10074.82], ["2025-12-05 10:32:53", 10059.82], ["2025-12-05 10:37:41", 10377.556], ["2025-12-05 10:37:44", 10206.172], ["2025-12-05 10:37:45", 10206.172], ["2025-12-05 11:54:47", 9720.082], ["2025-12-05 11:54:49", 10128.082]], "total_portfolio_

In [6]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 30.06,
  'timestamp': '2025-12-05 10:32:51',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 44.088,
  'timestamp': '2025-12-05 10:37:41',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 64.128,
  'timestamp': '2025-12-05 10:37:44',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 15.03,
  'timestamp': '2025-12-05 11:54:47',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [7]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [8]:
mcp_tools

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=None),
 Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=

In [9]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Vlad and my account is under the name Vlad. What's my balance and my holdings?"
model = gemini_model

In [10]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Your balance is 9540.082 and your holdings are {'AMZN': 12}.

### Now let's build our own MCP Client

In [11]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=None), Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=N

In [12]:
request = "My name is Vlad and my account is under the name Vlad. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Your balance is $9540.082. 


In [14]:
context = await read_accounts_resource("vlad")
print(context)

{"name": "vlad", "balance": 9540.082, "strategy": "", "holdings": {"AMZN": 12}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 30.06, "timestamp": "2025-12-05 10:32:51", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 44.088, "timestamp": "2025-12-05 10:37:41", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 64.128, "timestamp": "2025-12-05 10:37:44", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 15.03, "timestamp": "2025-12-05 11:54:47", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-05 10:32:51", 10074.82], ["2025-12-05 10:32:53", 10059.82], ["2025-12-05 10:37:41", 10377.556], ["2025-12-05 10:37:44", 10206.172], ["2025-12-05 10:37:45", 10206.172], ["2025-12-05 11:54:47", 9720.082], ["2025-12-05 11:54:49", 10128.082], ["2025-12-05 11:56:

In [15]:
from accounts import Account
Account.get("vlad").report()

'{"name": "vlad", "balance": 9540.082, "strategy": "", "holdings": {"AMZN": 12}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 30.06, "timestamp": "2025-12-05 10:32:51", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 44.088, "timestamp": "2025-12-05 10:37:41", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 64.128, "timestamp": "2025-12-05 10:37:44", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 15.03, "timestamp": "2025-12-05 11:54:47", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-05 10:32:51", 10074.82], ["2025-12-05 10:32:53", 10059.82], ["2025-12-05 10:37:41", 10377.556], ["2025-12-05 10:37:44", 10206.172], ["2025-12-05 10:37:45", 10206.172], ["2025-12-05 11:54:47", 9720.082], ["2025-12-05 11:54:49", 10128.082], ["2025-12-05 11:56

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>